# Quantitative Momentum Strategy
"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, I am going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, I will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

# Library Imports

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import percentileofscore as score
import xlsxwriter
import yfinance as yf
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Prepare the data

* Download latest stock price data
* Download 1 year back stock price data
* calculate the one year percentage change in stock price


In [6]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbol_list = sp500['Symbol'].unique().tolist()

end_date = '2024-06-05' # pd.to_datetime(datetime.today().strftime('%Y-%m-%d'))-pd.DateOffset(1)
start_date = pd.to_datetime(end_date)-pd.DateOffset(1)

df = yf.download(tickers=symbol_list, start=start_date, end=end_date).stack()

end_date_year_ago = pd.to_datetime(end_date) - pd.DateOffset(365)
start_date_year_ago = pd.to_datetime(start_date) - pd.DateOffset(365)

df_year_old = yf.download(tickers=symbol_list, start=start_date_year_ago, end=end_date_year_ago).stack()

[*********************100%%**********************]  503 of 503 completed
[*********************100%%**********************]  503 of 503 completed

3 Failed downloads:
['GEV', 'SOLV', 'VLTO']: YFChartError("%ticker%: Data doesn't exist for startDate = 1685937600, endDate = 1686024000")


In [7]:
def transform_dataframe(df):
    df.index.names = ['date', 'ticker']
    df.reset_index(level=0, drop=True, inplace=True)
    df.columns = df.columns.str.lower()
    df['Ticker'] = df.index
    df.reset_index(drop=True, inplace=True)
    df.drop(['close', 'open', 'high', 'low', 'volume'], axis=1)
    df.dropna()
    return df    

In [11]:
df = transform_dataframe(df)
df_year_old = transform_dataframe(df_year_old)

merged_df = df.merge(df_year_old, on='Ticker', suffixes=('_l', '_r'))
merged_df['One Year Price Return'] = ((merged_df['adj close_l']-merged_df['adj close_r']) / merged_df['adj close_r']) * 100
merged_df.rename(columns={'adj close_l':'Price'}, inplace=True)
final_df = merged_df[['One Year Price Return', 'Ticker', 'Price' ]]
final_df['Number of Shares to Buy'] = 'N/A'
final_df

Price,One Year Price Return,Ticker,Price,Number of Shares to Buy
0,10.561899,A,130.850006,N/A
1,-22.349766,AAL,11.500000,N/A
2,8.224749,AAPL,194.350006,N/A
3,18.471430,ABBV,162.139999,N/A
4,27.132854,ABNB,147.080002,N/A
...,...,...,...,...
495,29.144689,XYL,137.100006,N/A
496,5.493028,YUM,141.539993,N/A
497,-13.052808,ZBH,113.440002,N/A
498,12.507898,ZBRA,302.769989,N/A


# Removing Low-Momentum Stocks
The investment strategy that I am building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing is to remove all the stocks in our DataFrame that fall below this momentum threshold, and sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.

In [12]:
final_df.sort_values('One Year Price Return', ascending=False, inplace=True)
final_df = final_df[:50]
final_df.reset_index(inplace=True)
final_df

Price,index,One Year Price Return,Ticker,Price,Number of Shares to Buy
0,472,259.638572,VST,89.550003,N/A
1,412,244.992387,SMCI,771.609985,N/A
2,345,197.253050,NVDA,116.436996,N/A
3,340,134.993974,NRG,77.830002,N/A
4,88,132.133464,CEG,203.139999,N/A
5,133,119.620787,DECK,1066.829956,N/A
6,200,94.245102,GE,161.380005,N/A
7,480,89.589750,WDC,73.940002,N/A
8,488,88.846145,WRK,54.009998,N/A
9,283,87.410527,LLY,832.590027,N/A


# Calculating the Number of Shares to Buy
Now I will calculate the number of shares user need to buy. 

In [14]:
def portfolio_input():
    while True:
        try:
            return float(input("Enter size of your porfolio: "))
        except ValueError:
            print("That's not a number! Please enter number again")

In [15]:
position_size = portfolio_input() / len(final_df)
final_df['Number of Shares to Buy'] = position_size // final_df['Price']
final_df

Enter size of your porfolio:  10000000


Price,index,One Year Price Return,Ticker,Price,Number of Shares to Buy
0,472,259.638572,VST,89.550003,2233.0
1,412,244.992387,SMCI,771.609985,259.0
2,345,197.253050,NVDA,116.436996,1717.0
3,340,134.993974,NRG,77.830002,2569.0
4,88,132.133464,CEG,203.139999,984.0
5,133,119.620787,DECK,1066.829956,187.0
6,200,94.245102,GE,161.380005,1239.0
7,480,89.589750,WDC,73.940002,2704.0
8,488,88.846145,WRK,54.009998,3703.0
9,283,87.410527,LLY,832.590027,240.0


# Building a Better (and More Realistic) Momentum Strategy
Real-world quantitative investment firms differentiate between `high quality` and `low quality` momentum stocks:

* High-quality momentum stocks show `slow and steady` outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.
  
The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, I am going to build a strategy that selects stocks from the highest percentiles of:

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

In [16]:
def calculate_return(df, periods):
    returns = df.pct_change(periods=periods).shift(-periods)
    return returns

In [17]:
end_date = '2024-06-01'
start_date = pd.to_datetime(end_date) - pd.DateOffset(370)

df = yf.download(tickers=symbol_list, start=start_date, end=end_date).stack()
df.dropna()
df.index.names = ['date', 'ticker']
df.columns = df.columns.str.lower()

df_monthly = df.groupby('ticker').resample('M',level=0).last()

periods = {'One-Month Price Return': (1, '2024-04-30'),
           'Three-Month Price Return': (3, '2024-02-29'),
           'Six-Month Price Return': (6, '2023-11-30'),
           'Nine-Month Price Return': (9,'2023-08-31'),
           'One-Year Price Return': (12, '2023-05-31')}
final_df = pd.DataFrame()

final_df['ticker'] = symbol_list
final_df.set_index(['ticker'], inplace=True)

price = df_monthly.groupby('ticker')['adj close'].last().to_frame(name="Price")

final_df = final_df.merge(price, on='ticker', how='left')

for period_name, value in periods.items():
    period, date = value
    series_df = df_monthly.unstack(level=0)['adj close'].pct_change(periods=period).shift(-period).stack()[date].to_frame(name=period_name)
    final_df = final_df.merge(series_df, on='ticker', how='left')

final_df

[*********************100%%**********************]  503 of 503 completed


,Price,One-Month Price Return,Three-Month Price Return,Six-Month Price Return,Nine-Month Price Return,One-Year Price Return
ticker,,,,,,
MMM,100.139999,0.044819,0.309154,0.241575,0.174885,0.367197
AOS,83.639999,0.009657,0.012821,0.118541,0.168178,0.329984
ABT,102.190002,-0.035670,-0.134383,-0.010519,0.008398,0.022175
ABBV,161.240005,-0.008608,-0.075570,0.153878,0.129187,0.216186
ACN,282.290009,-0.061879,-0.243868,-0.146251,-0.117900,-0.063043
...,...,...,...,...,...,...
XYL,141.020004,0.081783,0.112861,0.348819,0.374014,0.424410
YUM,137.429993,-0.022263,-0.002273,0.105508,0.077856,0.088712
ZBRA,312.339996,-0.007057,0.117575,0.318002,0.135740,0.189549


# Calculating Momentum Percentiles
We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* One-Year Price Return
* Six-Month Price Return
* Three-Month Price Return
* One-Month Price Return

Here's how we'll do this:

In [18]:
time_periods = [ 'One-Year', 'Nine-Month', 'Six-Month','Three-Month','One-Month']

for period in time_periods:
    final_df[f'{period} Return Percentile'] = 0

final_df['Ticker'] = final_df.index
final_df.reset_index(drop=True, inplace=True)
final_df.dropna(inplace=True)

In [19]:
for period in time_periods:
    change_col = f'{period} Price Return'
    percentile_col = f'{period} Return Percentile'
    final_df[percentile_col] = final_df[change_col].apply(lambda x: score(final_df[change_col], x)/100)

final_df

,Price,One-Month Price Return,Three-Month Price Return,Six-Month Price Return,Nine-Month Price Return,One-Year Price Return,One-Year Return Percentile,Nine-Month Return Percentile,Six-Month Return Percentile,Three-Month Return Percentile,One-Month Return Percentile,Ticker
0,100.139999,0.044819,0.309154,0.241575,0.174885,0.367197,0.718,0.618,0.766,0.974,0.634,MMM
1,83.639999,0.009657,0.012821,0.118541,0.168178,0.329984,0.686,0.608,0.504,0.464,0.384,AOS
2,102.190002,-0.035670,-0.134383,-0.010519,0.008398,0.022175,0.242,0.306,0.200,0.102,0.162,ABT
3,161.240005,-0.008608,-0.075570,0.153878,0.129187,0.216186,0.548,0.540,0.598,0.230,0.292,ABBV
4,282.290009,-0.061879,-0.243868,-0.146251,-0.117900,-0.063043,0.142,0.124,0.052,0.018,0.098,ACN
...,...,...,...,...,...,...,...,...,...,...,...,...
498,141.020004,0.081783,0.112861,0.348819,0.374014,0.424410,0.774,0.882,0.890,0.780,0.820,XYL
499,137.429993,-0.022263,-0.002273,0.105508,0.077856,0.088712,0.328,0.434,0.480,0.408,0.226,YUM
500,312.339996,-0.007057,0.117575,0.318002,0.135740,0.189549,0.496,0.548,0.852,0.792,0.296,ZBRA
501,115.150002,-0.042650,-0.072318,-0.006133,-0.027471,-0.088736,0.120,0.234,0.210,0.240,0.142,ZBH


# Calculating the HQM Score

Now calculate `HQM Score`, which is the high-quality momentum score to use to filter for stocks in this investing strategy.

The `HQM Score` will be the `arithmetic mean` of the 4 momentum percentile scores.

In [20]:
final_df['HQM Score'] = final_df.apply(lambda row: pd.Series([row[f'{period} Return Percentile'] for period in time_periods]).mean(), axis=1)
final_df

,Price,One-Month Price Return,Three-Month Price Return,Six-Month Price Return,Nine-Month Price Return,One-Year Price Return,One-Year Return Percentile,Nine-Month Return Percentile,Six-Month Return Percentile,Three-Month Return Percentile,One-Month Return Percentile,Ticker,HQM Score
0,100.139999,0.044819,0.309154,0.241575,0.174885,0.367197,0.718,0.618,0.766,0.974,0.634,MMM,0.7420
1,83.639999,0.009657,0.012821,0.118541,0.168178,0.329984,0.686,0.608,0.504,0.464,0.384,AOS,0.5292
2,102.190002,-0.035670,-0.134383,-0.010519,0.008398,0.022175,0.242,0.306,0.200,0.102,0.162,ABT,0.2024
3,161.240005,-0.008608,-0.075570,0.153878,0.129187,0.216186,0.548,0.540,0.598,0.230,0.292,ABBV,0.4416
4,282.290009,-0.061879,-0.243868,-0.146251,-0.117900,-0.063043,0.142,0.124,0.052,0.018,0.098,ACN,0.0868
...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,141.020004,0.081783,0.112861,0.348819,0.374014,0.424410,0.774,0.882,0.890,0.780,0.820,XYL,0.8292
499,137.429993,-0.022263,-0.002273,0.105508,0.077856,0.088712,0.328,0.434,0.480,0.408,0.226,YUM,0.3752
500,312.339996,-0.007057,0.117575,0.318002,0.135740,0.189549,0.496,0.548,0.852,0.792,0.296,ZBRA,0.5968
501,115.150002,-0.042650,-0.072318,-0.006133,-0.027471,-0.088736,0.120,0.234,0.210,0.240,0.142,ZBH,0.1892


# Selecting the 50 Best Momentum Stocks

In [21]:
final_df.sort_values('HQM Score', ascending=False, inplace=True)
final_df = final_df[:50]
final_df.reset_index(inplace=True, drop=True)
final_df

,Price,One-Month Price Return,Three-Month Price Return,Six-Month Price Return,Nine-Month Price Return,One-Year Price Return,One-Year Return Percentile,Nine-Month Return Percentile,Six-Month Return Percentile,Three-Month Return Percentile,One-Month Return Percentile,Ticker,HQM Score
0,99.080002,0.306435,0.822921,1.823669,2.201940,3.231567,1.000,1.000,0.998,1.000,0.996,VST,0.9988
1,109.633003,0.268871,0.385860,1.344404,1.221798,1.898667,0.996,0.996,0.996,0.986,0.986,NVDA,0.9920
2,1093.920044,0.336543,0.221452,0.647544,1.067550,1.302990,0.990,0.990,0.984,0.948,0.998,DECK,0.9820
3,217.250000,0.170174,0.294210,0.801138,1.097877,1.607704,0.994,0.992,0.992,0.966,0.962,CEG,0.9812
4,84.650002,0.268944,0.272758,0.611668,0.715414,0.986719,0.982,0.982,0.980,0.960,0.988,HWM,0.9784
5,204.050003,0.235371,0.298474,0.595714,0.809452,0.840261,0.966,0.986,0.976,0.968,0.982,QCOM,0.9756
6,81.000000,0.114628,0.472331,0.715491,1.205326,1.475539,0.992,0.994,0.988,0.994,0.900,NRG,0.9736
7,125.000000,0.106586,0.380870,0.645930,0.794355,0.843597,0.968,0.984,0.982,0.984,0.888,MU,0.9612
8,120.430000,0.178260,0.357699,0.331691,0.597346,0.858699,0.970,0.970,0.874,0.980,0.968,NTAP,0.9524
9,132.369995,0.096050,0.214180,0.460947,0.507785,0.770718,0.948,0.942,0.966,0.944,0.862,APH,0.9324


# Calculating the Number of Shares to Buy

In [22]:
position_size = portfolio_input() / len(final_df.index)
final_df['Number of Shares to Buy'] = position_size // final_df['Price']
final_df

Enter size of your porfolio:  750000000


,Price,One-Month Price Return,Three-Month Price Return,Six-Month Price Return,Nine-Month Price Return,One-Year Price Return,One-Year Return Percentile,Nine-Month Return Percentile,Six-Month Return Percentile,Three-Month Return Percentile,One-Month Return Percentile,Ticker,HQM Score,Number of Shares to Buy
0,99.080002,0.306435,0.822921,1.823669,2.201940,3.231567,1.000,1.000,0.998,1.000,0.996,VST,0.9988,151392.0
1,109.633003,0.268871,0.385860,1.344404,1.221798,1.898667,0.996,0.996,0.996,0.986,0.986,NVDA,0.9920,136820.0
2,1093.920044,0.336543,0.221452,0.647544,1.067550,1.302990,0.990,0.990,0.984,0.948,0.998,DECK,0.9820,13712.0
3,217.250000,0.170174,0.294210,0.801138,1.097877,1.607704,0.994,0.992,0.992,0.966,0.962,CEG,0.9812,69044.0
4,84.650002,0.268944,0.272758,0.611668,0.715414,0.986719,0.982,0.982,0.980,0.960,0.988,HWM,0.9784,177200.0
5,204.050003,0.235371,0.298474,0.595714,0.809452,0.840261,0.966,0.986,0.976,0.968,0.982,QCOM,0.9756,73511.0
6,81.000000,0.114628,0.472331,0.715491,1.205326,1.475539,0.992,0.994,0.988,0.994,0.900,NRG,0.9736,185185.0
7,125.000000,0.106586,0.380870,0.645930,0.794355,0.843597,0.968,0.984,0.982,0.984,0.888,MU,0.9612,120000.0
8,120.430000,0.178260,0.357699,0.331691,0.597346,0.858699,0.970,0.970,0.874,0.980,0.968,NTAP,0.9524,124553.0
9,132.369995,0.096050,0.214180,0.460947,0.507785,0.770718,0.948,0.942,0.966,0.944,0.862,APH,0.9324,113318.0


# Formatting Our Excel Output

In [23]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
final_df.to_excel(writer, sheet_name='Momentum Strategy', index = False)

# Creating the Formats For .xlsx File

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase


In [24]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [25]:
column_formats = { 
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', integer_template],
    'E': ['One-Year Return Percentile', integer_template],
    'F': ['Six-Month Price Return', integer_template],
    'G': ['Six-Month Return Percentile', integer_template],
    'H': ['Three-Month Price Return', integer_template],
    'I': ['Three-Month Return Percentile', integer_template],
    'J': ['One-Month Price Return', integer_template],
    'K': ['One-Month Return Percentile', integer_template],
    'L': ['HQM Score', integer_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

writer.close()